# **Reduces Dataset to 5 Classes**

In [ ]:
import os
import shutil

In [ ]:
# Mounts Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzips the dataset
!unzip /content/drive/MyDrive/simpsons_dataset.zip

# Stores a path to the dataset
dataset_dir = '/content/simpsons_dataset'

# Stores a list of all the classes in the dataset
all_classes = os.listdir(dataset_dir)

# Stores a list of 5 classes to keep
classes_to_keep = ['homer_simpson', 'bart_simpson', 'moe_szyslak',
                   'krusty_the_clown', 'apu_nahasapeemapetilon']

# Iterates through all the classes and removes the ones not to keep
for class_name in all_classes:
    class_dir = os.path.join(dataset_dir, class_name)

    if os.path.isdir(class_dir) and class_name not in classes_to_keep:
        shutil.rmtree(class_dir)

# **Prepares Data**

In [ ]:
import tensorflow as tf

In [ ]:
# Creates an ImageDataGenerator to prepare the images for training and testing
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,         # Normalizes pixel values to [0, 1]
    validation_split = 0.2    # Reserves 20% of images for testing
)

# Uses the ImageDataGenerator to load and prepare images for training
train_generator = datagen.flow_from_directory(
    dataset_dir,                 # Path to dataset
    target_size = (224, 224),    # Resizes images to 224x224
    class_mode = 'categorical',  # For multi-class classification
    subset = 'training',         # Use for training
    batch_size = 64              # Sets batch size to 64
)

# Uses the ImageDataGenerator to load and prepare images for testing
test_generator = datagen.flow_from_directory(
    dataset_dir,                 # Path to dataset
    target_size = (224, 224),    # Resizes images to 224x224
    class_mode = 'categorical',  # For multi-class classification
    subset = 'validation'        # Use for testing
)

#**Builds, Trains & Tests your CNN**

In [ ]:
# Builds the model
CNN_model = tf.keras.Sequential()
CNN_model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu', padding='same'))
CNN_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
CNN_model.add(tf.keras.layers.MaxPooling2D())
CNN_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
CNN_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
CNN_model.add(tf.keras.layers.MaxPooling2D())
CNN_model.add(tf.keras.layers.Flatten())
CNN_model.add(tf.keras.layers.Dense(512, activation='relu'))
CNN_model.add(tf.keras.layers.Dense(5, activation='softmax'))

# Compiles the model
sgd = tf.keras.optimizers.SGD(0.01, momentum=0.9, decay=0.01/20, nesterov=False)
CNN_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Trains the model
train_history = CNN_model.fit(train_generator, epochs=20)

# Evaluates the model on the test data
loss, accuracy = CNN_model.evaluate(test_generator)
print(f'Test model loss: {round(loss,4)}')
print(f'Test model accuracy: {round(100*accuracy,2)}%')

#**Plots Parameters Table**

In [ ]:
# Plots a parameters table
CNN_model.summary()

#**Applies the Model**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Creates an ImageDataGenerator to prepare the images for the model
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Uses the ImageDataGenerator to load and prepare images for the model
prediction_data_generator = datagen.flow_from_directory(
    '/content/simpsons_dataset',  # Path to dataset
    target_size=(224, 224),       # Resizes images to 224x224
    class_mode=None,              # For making predictions
    shuffle=True                  # Shuffles images
)

# Maps class indexes to class labels
class_indices = prediction_data_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

In [ ]:
# Creates a 20x10 figure with 4 rows and 8 columns
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
axes = axes.flatten()

# Loops through 32 images
for i in range(32):
    # Gets the next image
    image = next(prediction_data_generator)

    # Predicts the character in the image
    prediction = CNN_model.predict(image, verbose = 0)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Plots the image and its predicted character within the figure
    ax = axes[i]
    ax.imshow(image[0])
    ax.set_title(f'{class_labels[predicted_class]}')
    ax.axis('off')

# Displays the figure
plt.tight_layout()
plt.show()